# Chapter 8: Data Wrangling: Join, Combine, and Reshape

In many applications, data may be spread across a number of files or be arranged in a form that is not easy to analyze. This chapter focuses on tools to help combine, join, and rearrange data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## I. Merging Datasets

### 1. Default merge operation for data frames

In [2]:
# Generate two data frames
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


`df1.merge(df2)` merges df1 with df2:

In [4]:
df1.merge(df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
# It is the same as df2.merge(df1)
df2.merge(df1)

,key,data2,data1
0,a,0,2
1,a,0,4
2,a,0,5
3,b,1,0
4,b,1,1
5,b,1,6


In [6]:
pd.merge(df2, df1)

,key,data2,data1
0,a,0,2
1,a,0,4
2,a,0,5
3,b,1,0
4,b,1,1
5,b,1,6


Q: Can you identify the rule followed by merge?

In [7]:
df3 = pd.DataFrame({'key': ['a', 'b', 'b'],
                    'data2': range(3)})
df3

,key,data2
0,a,0
1,b,1
2,b,2


In [8]:
# Can you predict the resulting data frame?
df1.merge(df3)

,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


**It is a good practice to specify explicitly which column(s) to join on.**

In [9]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [10]:
df1.merge(df3, on='key')

,key,data1,data2
0,b,0,1
1,b,0,2
2,b,1,1
3,b,1,2
4,b,6,1
5,b,6,2
6,a,2,0
7,a,4,0
8,a,5,0


### 2. What if the column to join has different names in the two data frames?

In [11]:
homework = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare'],
    'Hw1': [100, 90, 80],
    'Hw2': [60, 70, 80]
})
homework

,Name,Hw1,Hw2
0,Alice,100,60
1,Bob,90,70
2,Clare,80,80


In [12]:
exam = pd.DataFrame({
    "Full Name": ['Alice', 'Bob', 'Clare'],
    "Midterm": [70, 80, 90],
    "Final": [85, 65, 75]
})
exam

,Full Name,Midterm,Final
0,Alice,70,85
1,Bob,80,65
2,Clare,90,75


In [13]:
pd.merge(homework, exam, left_on="Name", right_on="Full Name")

,Name,Hw1,Hw2,Full Name,Midterm,Final
0,Alice,100,60,Alice,70,85
1,Bob,90,70,Bob,80,65
2,Clare,80,80,Clare,90,75


### 3. What if the column to join has different values?

In [14]:
homework = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Hw1': [100, 90, 80, 70],
    'Hw2': [60, 70, 80, 90]
})
homework

,Name,Hw1,Hw2
0,Alice,100,60
1,Bob,90,70
2,Clare,80,80
3,David,70,90


In [15]:
exam = pd.DataFrame({
    "Full Name": ['Alice', 'Bob', 'Clare', 'Eli'],
    "Midterm": [70, 80, 90, 100],
    "Final": [85, 65, 75, 55]
})
exam

,Full Name,Midterm,Final
0,Alice,70,85
1,Bob,80,65
2,Clare,90,75
3,Eli,100,55


In [16]:
pd.merge(homework, exam, left_on="Name", right_on="Full Name")

,Name,Hw1,Hw2,Full Name,Midterm,Final
0,Alice,100,60,Alice,70,85
1,Bob,90,70,Bob,80,65
2,Clare,80,80,Clare,90,75


Different join types with `how` argument
- inner: Use only the keys combinations observed in both tables
- outer: Use all possible keys combinations
- left: Use all keys found in the first data frame
- right: Use all keys found in the second data frame

In [17]:
pd.merge(homework, exam, left_on="Name", right_on="Full Name",
         how='outer')

,Name,Hw1,Hw2,Full Name,Midterm,Final
0,Alice,100.0,60.0,Alice,70.0,85.0
1,Bob,90.0,70.0,Bob,80.0,65.0
2,Clare,80.0,80.0,Clare,90.0,75.0
3,David,70.0,90.0,NaN,NaN,NaN
4,NaN,NaN,NaN,Eli,100.0,55.0


### 4. What if we want to join on multiple columns?

In [18]:
homework = pd.DataFrame({
    'Semester': ['Fall 2018', 'Fall 2018', 'Fall 2019', 'Fall 2019'],
    'Name': ['Alice', 'Bob', 'Clare', 'Alice'],
    'Hw1': [50, 90, 80, 70],
    'Hw2': [60, 70, 80, 90]
})
homework

,Semester,Name,Hw1,Hw2
0,Fall 2018,Alice,50,60
1,Fall 2018,Bob,90,70
2,Fall 2019,Clare,80,80
3,Fall 2019,Alice,70,90


In [19]:
exam = pd.DataFrame({
    'When': ['Fall 2018', 'Fall 2018', 'Fall 2019', 'Fall 2019'],
    "Name": ['Alice', 'Bob', 'Clare', 'Alice'],
    "Midterm": [60, 80, 90, 100],
    "Final": [45, 65, 75, 55]
})
exam

,When,Name,Midterm,Final
0,Fall 2018,Alice,60,45
1,Fall 2018,Bob,80,65
2,Fall 2019,Clare,90,75
3,Fall 2019,Alice,100,55


In [20]:
pd.merge(homework, exam, on='Name')

,Semester,Name,Hw1,Hw2,When,Midterm,Final
0,Fall 2018,Alice,50,60,Fall 2018,60,45
1,Fall 2018,Alice,50,60,Fall 2019,100,55
2,Fall 2019,Alice,70,90,Fall 2018,60,45
3,Fall 2019,Alice,70,90,Fall 2019,100,55
4,Fall 2018,Bob,90,70,Fall 2018,80,65
5,Fall 2019,Clare,80,80,Fall 2019,90,75


In [21]:
pd.merge(homework, exam, left_on=['Semester', 'Name'],
         right_on=['When', 'Name'])

,Semester,Name,Hw1,Hw2,When,Midterm,Final
0,Fall 2018,Alice,50,60,Fall 2018,60,45
1,Fall 2018,Bob,90,70,Fall 2018,80,65
2,Fall 2019,Clare,80,80,Fall 2019,90,75
3,Fall 2019,Alice,70,90,Fall 2019,100,55


### 5. What if there are overlapping columns?

In [22]:
homework = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Hw1': [100, 90, 80, 70],
    'Hw2': [60, 70, 80, 90],
    'Average': [80, 80, 80, 80]
})
homework

,Name,Hw1,Hw2,Average
0,Alice,100,60,80
1,Bob,90,70,80
2,Clare,80,80,80
3,David,70,90,80


In [23]:
exam = pd.DataFrame({
    "Name": ['Alice', 'Bob', 'Clare', 'Eva'],
    "Midterm": [60, 80, 90, 100],
    "Final": [45, 65, 75, 55],
    "Average": [52.5, 72.5, 82.5, 77.5]
})
exam

,Name,Midterm,Final,Average
0,Alice,60,45,52.5
1,Bob,80,65,72.5
2,Clare,90,75,82.5
3,Eva,100,55,77.5


In [24]:
pd.merge(homework, exam, on='Name', how='outer')

,Name,Hw1,Hw2,Average_x,Midterm,Final,Average_y
0,Alice,100.0,60.0,80.0,60.0,45.0,52.5
1,Bob,90.0,70.0,80.0,80.0,65.0,72.5
2,Clare,80.0,80.0,80.0,90.0,75.0,82.5
3,David,70.0,90.0,80.0,NaN,NaN,NaN
4,Eva,NaN,NaN,NaN,100.0,55.0,77.5


In [25]:
pd.merge(homework, exam, on='Name', suffixes=('_hw', '_ex'), how='outer')

,Name,Hw1,Hw2,Average_hw,Midterm,Final,Average_ex
0,Alice,100.0,60.0,80.0,60.0,45.0,52.5
1,Bob,90.0,70.0,80.0,80.0,65.0,72.5
2,Clare,80.0,80.0,80.0,90.0,75.0,82.5
3,David,70.0,90.0,80.0,NaN,NaN,NaN
4,Eva,NaN,NaN,NaN,100.0,55.0,77.5


### 6. What if we want to merge on index?

In [26]:
homework = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Hw1': [100, 90, 80, 70],
    'Hw2': [60, 70, 80, 90],
    'Average': [80, 80, 80, 80]
}, index=[111, 222, 333, 444])
homework

,Name,Hw1,Hw2,Average
111,Alice,100,60,80
222,Bob,90,70,80
333,Clare,80,80,80
444,David,70,90,80


In [27]:
exam = pd.DataFrame({
    "Name": ['Alice', 'Bob', 'Clare', 'Eva'],
    "Midterm": [60, 80, 90, 100],
    "Final": [45, 65, 75, 55],
    "Average": [52.5, 72.5, 82.5, 77.5]
})
exam = exam.set_index('Name')
exam

,Midterm,Final,Average
Name,,,
Alice,60,45,52.5
Bob,80,65,72.5
Clare,90,75,82.5
Eva,100,55,77.5


In [28]:
pd.merge(homework, exam, left_on='Name', right_index=True)

,Name,Hw1,Hw2,Average_x,Midterm,Final,Average_y
111,Alice,100,60,80,60,45,52.5
222,Bob,90,70,80,80,65,72.5
333,Clare,80,80,80,90,75,82.5


## II. Concatenations

### 1. Concatenating NumPy Arrays
My personal favorite methods are np.hstack() for horizontal concatenation and np.vstack() for vertical concatenation.

In [29]:
arr1 = np.arange(12).reshape([3, 4])
print(arr1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [30]:
arr2 = np.arange(10, 90, 10).reshape([2, 4])
print(arr2)

[[10 20 30 40]
 [50 60 70 80]]


In [31]:
print(np.vstack([arr1, arr2]))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [10 20 30 40]
 [50 60 70 80]]


In [32]:
arr3 = np.arange(100, 10, -10).reshape([3, 3])
print(arr3)

[[100  90  80]
 [ 70  60  50]
 [ 40  30  20]]


In [33]:
print(np.hstack([arr1, arr3]))

[[  0   1   2   3 100  90  80]
 [  4   5   6   7  70  60  50]
 [  8   9  10  11  40  30  20]]


### 2. Concatenating Data Frames

In [34]:
spring_records = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Clare', 'David'],
    'Homework': [60, 70, 80, 90],
    'Exam': [65, 75, 85, 95]
})
spring_records

,Name,Homework,Exam
0,Alice,60,65
1,Bob,70,75
2,Clare,80,85
3,David,90,95


In [35]:
fall_records = pd.DataFrame({
    'Name': ['Alice', 'Eva', 'Fred', 'Gabriel'],
    'Homework': [66, 77, 88, 99],
    'Exam': [69, 79, 89, 99]
})
fall_records

,Name,Homework,Exam
0,Alice,66,69
1,Eva,77,79
2,Fred,88,89
3,Gabriel,99,99


In [36]:
pd.concat([spring_records, fall_records])

,Name,Homework,Exam
0,Alice,60,65
1,Bob,70,75
2,Clare,80,85
3,David,90,95
0,Alice,66,69
1,Eva,77,79
2,Fred,88,89
3,Gabriel,99,99


In [37]:
pd.concat([spring_records, fall_records], axis=1)

,Name,Homework,Exam,Name,Homework,Exam
0,Alice,60,65,Alice,66,69
1,Bob,70,75,Eva,77,79
2,Clare,80,85,Fred,88,89
3,David,90,95,Gabriel,99,99


## Example: Analyzing Airport Operations

Download `airports.csv`, `airport-frequencies.csv`, `countries.csv`, `regions.csv` from [OurAirports.com](https://ourairports.com/data/)

In [38]:
import numpy as np
import pandas as pd

In [58]:
url = "https://ourairports.com/data/airports.csv"
airports = pd.read_csv(url, delimiter=",")
url = "https://ourairports.com/data/airport-frequencies.csv"
airports_freqs = pd.read_csv(url, delimiter=",")
url = "https://ourairports.com/data/countries.csv"
countries = pd.read_csv(url, delimiter=",")
regions = pd.read_csv(url, delimiter=",")

In [59]:
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [60]:
airports_freqs.head()

,id,airport_ref,airport_ident,type,description,frequency_mhz
0,70518,6528,00CA,CTAF,CTAF,122.9
1,307581,6589,01FL,ARCAL,NaN,122.9
2,75239,6589,01FL,CTAF,CEDAR KNOLL TRAFFIC,122.8
3,60191,6756,04CA,CTAF,CTAF,122.9
4,59287,6779,04MS,UNIC,UNICOM,122.8


In [61]:
countries.head()

,id,code,name,continent,wikipedia_link,keywords
0,302672,AD,Andorra,EU,https://en.wikipedia.org/wiki/Andorra,NaN
1,302618,AE,United Arab Emirates,AS,https://en.wikipedia.org/wiki/United_Arab_Emir...,"UAE,مطارات في الإمارات العربية المتحدة"
2,302619,AF,Afghanistan,AS,https://en.wikipedia.org/wiki/Afghanistan,NaN
3,302722,AG,Antigua and Barbuda,NaN,https://en.wikipedia.org/wiki/Antigua_and_Barbuda,NaN
4,302723,AI,Anguilla,NaN,https://en.wikipedia.org/wiki/Anguilla,NaN


In [62]:
regions.head()

,id,code,name,continent,wikipedia_link,keywords
0,302672,AD,Andorra,EU,https://en.wikipedia.org/wiki/Andorra,NaN
1,302618,AE,United Arab Emirates,AS,https://en.wikipedia.org/wiki/United_Arab_Emir...,"UAE,مطارات في الإمارات العربية المتحدة"
2,302619,AF,Afghanistan,AS,https://en.wikipedia.org/wiki/Afghanistan,NaN
3,302722,AG,Antigua and Barbuda,NaN,https://en.wikipedia.org/wiki/Antigua_and_Barbuda,NaN
4,302723,AI,Anguilla,NaN,https://en.wikipedia.org/wiki/Anguilla,NaN


#### 1. Select data with multiple conditions

In [66]:
# Find the region code for New York from region data frame.

index = (regions['name'] == "New York")
code = regions[index]['local_code'].values()
print(code)

In [ ]:
# Extract all large airports in New York state from airports data frame



In [ ]:
# Extract the name, identification code, and municipality of 
# all airports with ISO region "US-NY" and type "large_airport"



#### 2. Sorting

In [ ]:
# From airport_freq, extract all communication frequencies for KJFK,
# with frequencies sorted in ascending order



In [ ]:
# From airport_freq, extract all communication frequencies for KJFK,
# with frequencies sorted in descending order



#### 3. Filter on a list of values

In [ ]:
# Extract all communication frequencies used for a large airport in New York state



#### 4. Grouping

In [ ]:
# Calculate the number of large airports for each country



In [ ]:
# Find the top 5 countries having the largest amount of large airports



#### 5. Merging

In [ ]:
# Merge the above result with countries data frame to find the name of the countries



In [ ]:
# Append full country name and region name to airports.

